In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from numpy import arange, meshgrid, exp, cos, pi, sqrt, e

## Source

![figure_5.png](../assets/figure_5.png)

In [ ]:
class Source():
    def __init__(self, resolution):
        self.resolution = resolution
        self.signal = 1
        self.data = None
        self.ub = None
        self.lb = None

    def generate_arena(self, epoch):
        """To be run at every reset. This will generate data for the sampler."""
        ele = 300
        self.lb = [-ele, -ele]
        self.ub = [ele, ele]
        self.data = self.get_data(self.lb, self.ub, self.resolution, self.signal)
    
    def get_data(self, lb = [-5, -5], ub = [5, 5], resolution=100, signal=1):
        """
        input:
        -lb: Lower bound for the env.
        -ub: Upper bound for the env.
        -resolution: Shape of the functional space (resolution, resolution) 
        -signal: Decides which source environment to use. 1 for the unimodal and 0 for the multimodal.

        output:
        -data: Array of shape (resolution**2, 3) containing locations X1, X2 and the signal measurements Y
        """
        x1 = np.linspace(lb[0], ub[0], resolution)
        x2 = np.linspace(lb[1], ub[1], resolution)
        X1, X2 = np.meshgrid(x1, x2)
        # Y, self.a, self.b = self.unimodal_signal(X1, X2) if signal == 1 else self.multimodal_signal(X1, X2)
        data = np.hstack((X1.reshape(-1, 1), X2.reshape(-1, 1)))
        # print("Data_ :", data[2][2])
        return data



    def get_info(self):
        return self.data, self.lb, self.ub

: 

## simple regression

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=11, padding=5),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=9, padding=4),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
            nn.LeakyReLU(),
        )

        # self.avg_pool = nn.AdaptiveAvgPool2d((25, 25))

        self.fc = nn.Sequential(
            nn.Linear(8*25*25, 5000),
            nn.LeakyReLU(),
            nn.Linear(5000, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 100*2)
        )

    def forward(self, x):
        x = self.conv(x)
        # x = self.avg_pool(x)
        # print(x.shape)
        # os.system('pause')
        temp = x.shape[0]
        x = torch.flatten(x, start_dim=1)
        # print(x.shape, vector.shape)
        # os.system('pause')
        x = self.fc(x)

        return x.view(temp, 100, 2)